In [17]:
pip install pandas sqlalchemy pyodbc

  Using cached sqlalchemy-2.0.44-py3-none-any.whl.metadata (9.5 kB)
  Using cached typing_extensions-4.15.0-py3-none-any.whl.metadata (3.3 kB)
Using cached sqlalchemy-2.0.44-py3-none-any.whl (1.9 MB)
Using cached typing_extensions-4.15.0-py3-none-any.whl (44 kB)

   ---------------------------------------- 0/3 [typing-extensions]
   ---------------------------------------- 0/3 [typing-extensions]
   ---------------------------------------- 0/3 [typing-extensions]
   ------------- -------------------------- 1/3 [greenlet]
   ------------- -------------------------- 1/3 [greenlet]
   ------------- -------------------------- 1/3 [greenlet]
   ------------- -------------------------- 1/3 [greenlet]
   ------------- -------------------------- 1/3 [greenlet]
   ------------- -------------------------- 1/3 [greenlet]
   ------------- -------------------------- 1/3 [greenlet]
   ------------- -------------------------- 1/3 [greenlet]
   ------------- -------------------------- 1/3 [greenlet]
 


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [18]:
#imports


import pandas as pd
import pyodbc
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

from sqlalchemy import create_engine
from sqlalchemy.exc import SQLAlchemyError
from urllib.parse import quote_plus


In [6]:
# Download VADER lexicon for sentiment analysis if not already present.
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...


True

In [28]:


def fetch_data_from_sql_alchemy():

    driver = "ODBC Driver 17 for SQL Server"
    server = "localhost"
    database = "MarketingAnalytics_db"
    
    #Link of connecting to the  msql server 
    connection_string = f"mssql+pyodbc:///?odbc_connect=Driver={quote_plus(driver)};Server={server};Database={database};Trusted_Connection=yes;"
    # SQL  query to  retrieve the data from  msql  data
    query = "SELECT ReviewID, CustomerID, ProductID, ReviewDate, Rating, ReviewText FROM customer_reviews"

    try:
        # Create the SQLAlchemy Engine
        engine = create_engine(connection_string)
     
        # Connect and Fetch Data
    
        with engine.connect() as conn:
             df = pd.read_sql(query, conn)
        
        # Return the fetched data as a DataFrame
        return df

    except SQLAlchemyError as e:
        print(f"A SQLAlchemy error occurred: {e}")
        # Return an empty DataFrame on failure
        return pd.DataFrame()
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return pd.DataFrame()



In [29]:
#  the function  to check if we have the data 
reviews_df = fetch_data_from_sql_alchemy()
print(reviews_df.head())

C:\Users\PC\AppData\Local\Temp\ipykernel_8436\1678901048.py:18: SAWarning: Unrecognized server version info '17.0.1000.7'.  Some SQL Server features may not function properly.
  with engine.connect() as conn:


   ReviewID  CustomerID  ProductID  ReviewDate  Rating  \
0         1          77         18  2023-12-23       3   
1         2          80         19  2024-12-25       5   
2         3          50         13  2025-01-26       4   
3         4          78         15  2025-04-21       3   
4         5          64          2  2023-07-16       3   

                                 ReviewText  
0   Average  experience,  nothing  special.  
1            The  quality  is    top-notch.  
2   Five  stars  for  the  quick  delivery.  
3  Good  quality,  but  could  be  cheaper.  
4   Average  experience,  nothing  special.  


In [30]:
# Initialize the VADER sentiment intensity analyzer for analyzing the sentiment of text data
sia = SentimentIntensityAnalyzer()

In [31]:
def calculate_sentiment(review):
    # Get the sentiment scores for the review text
    sentiment = sia.polarity_scores(review)
    # Return the compound score, which is a normalized score between -1 (most negative) and 1 (most positive)
    return sentiment['compound']

In [ ]:
# Defining a function to categorize sentiment using both the sentiment score and the review rating
def categorize_sentiment(score, rating):

# Positive sentiment score
    if score > 0.05:  
        if rating >= 4:
            return 'Positive'  
        elif rating == 3:
            return 'Mixed Positive'  
        else:
            return 'Mixed Negative' 
# Negative sentiment score        
    elif score < -0.05:  
        if rating <= 2:
            return 'Negative' 
        elif rating == 3:
            return 'Mixed Negative' 
        else:
            return 'Mixed Positive'  
        
# Neutral sentiment score        
    else:  
        if rating >= 4:
            return 'Positive'  
        elif rating <= 2:
            return 'Negative'  
        else:
            return 'Neutral'  


In [37]:
# A function to bucket sentiment scores into text ranges
def sentiment_bucket(score):
    if score >= 0.5:
        return '0.5 to 1.0' 
    elif 0.0 <= score < 0.5:
        return '0.0 to 0.49' 
    elif -0.5 <= score < 0.0:
        return '-0.49 to 0.0' 
    else:
        return '-1.0 to -0.5'

In [40]:
# Apply sentiment analysis to calculate sentiment scores for each review
reviews_df['SentimentScore'] = reviews_df['ReviewText'].apply(calculate_sentiment)

In [41]:
# Apply sentiment categorization using both text and rating
reviews_df['SentimentCategory'] = reviews_df.apply(
    lambda row: categorize_sentiment(row['SentimentScore'], row['Rating']), axis=1)

In [42]:
# Apply sentiment bucketing to categorize scores into defined ranges
reviews_df['SentimentBucket'] = reviews_df['SentimentScore'].apply(sentiment_bucket)

In [43]:
# Display the first few rows of the DataFrame with sentiment scores, categories, and buckets
print(reviews_df.head())

   ReviewID  CustomerID  ProductID  ReviewDate  Rating  \
0         1          77         18  2023-12-23       3   
1         2          80         19  2024-12-25       5   
2         3          50         13  2025-01-26       4   
3         4          78         15  2025-04-21       3   
4         5          64          2  2023-07-16       3   

                                 ReviewText  SentimentScore SentimentCategory  \
0   Average  experience,  nothing  special.         -0.3089    Mixed Negative   
1            The  quality  is    top-notch.          0.0000          Positive   
2   Five  stars  for  the  quick  delivery.          0.0000          Positive   
3  Good  quality,  but  could  be  cheaper.          0.2382    Mixed Positive   
4   Average  experience,  nothing  special.         -0.3089    Mixed Negative   

  SentimentBucket  
0    -0.49 to 0.0  
1     0.0 to 0.49  
2     0.0 to 0.49  
3     0.0 to 0.49  
4    -0.49 to 0.0  


In [44]:
# Save the DataFrame with sentiment scores, categories, and buckets to a new CSV file
reviews_df.to_csv('Fact_customer_reviews_with_sentiment.csv', index=False)